<a href="https://colab.research.google.com/github/marcelohpos/PROJETO-ICD/blob/master/Dados_e_Tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependência (só precisa rodar uma vez)
!pip install geopy seaborn

In [2]:
!pip install calmap

In [3]:
# Importar bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import numpy as np
import scipy.stats as st

# Dados e Tratamento

## **Contexto**

Entre os dias 10 e 21 de novembro de 2025, realizar-se-á a COP30 (Conferência das Partes) na cidade de Belém, Pará, Brasil. Neste contexto de reunião entre diferentes nações em torno do assunto das mudanças climáticas, entendemos ser importante ter um vislumbre de nossa situação atual - em que posição o Brasil se encontra no contexto climático, quais pontos são mais sensíveis e demandam ações governamentais e quais o país vêm dando exemplo ao mundo.

## **Fontes dos Dados**

Infelizmente, não logramos encontrar bases de dados para um número considerável de cidades da América Latina, porém, encontramos dados coletados em anos recentes para as capitais latinoamericanas, de forma que as bases se configuram da seguinte forma:
- **Latin America Weather and Air Quality Data**: Consiste num dataset contendo dois arquivos CSV gerados a partir da API Open-Meteo, contendo informação relativa a vários países do continente latinoamericano. Estes dados incluem o nome do país, sua capital, e dados climáticos e de qualidade do ar.

**Fonte:** [Latin America Weather and Air Quality Data - Kaggle](https://www.kaggle.com/datasets/anycaroliny/latin-america-weather-and-air-quality-data?select=LA_daily_climate.csv)


In [4]:
# Carregar os dados
url1 = "https://raw.githubusercontent.com/marcelohpos/PROJETO-ICD/master/datasets/LA_daily_air_quality.csv"
dados_clima = pd.read_csv(url1)

url2 = "https://raw.githubusercontent.com/marcelohpos/PROJETO-ICD/master/datasets/LA_daily_climate.csv"
dados_temp = pd.read_csv(url2)

### **Traduzindo informações da base**

Como a base de dados está em inglês, traduzimos os nomes das colunas para o português brasileiro.

In [5]:
dados_clima.columns = ['Data', 'Latitude', 'Longitude', 'PM10', 'PM2_5', 'Monóxido de Carbono (CO)', 'Dióxido de Nitrogênio (NO2)', 'Dioxido de Enxofre(SO2)', 'Ozônio(O3)']

dados_temp.columns = ['País','Cidade','Data','Latitude','Longitude','Temperatura_2m_max','Temperatura_2m_min','Temperatura_2m_media','Temperatura_aparente_max','Temperatura_aparente_min','Temperatura_aparente_media','Soma_precipitacao','Velocidade_vento_10m_max','Et0_fao_evapotranspiração']

### **Visualização dos Dados**

In [6]:
dados_temp

,País,Cidade,Data,Latitude,Longitude,Temperatura_2m_max,Temperatura_2m_min,Temperatura_2m_media,Temperatura_aparente_max,Temperatura_aparente_min,Temperatura_aparente_media,Soma_precipitacao,Velocidade_vento_10m_max,Et0_fao_evapotranspiração
0,Argentina,Buenos Aires,2020-01-01 00:00:00+00:00,-34.6037,-58.3816,24.767500,19.267500,21.973753,25.061949,18.268500,22.049086,0.6,24.336640,4.160744
1,Argentina,Buenos Aires,2020-01-02 00:00:00+00:00,-34.6037,-58.3816,24.217500,18.317501,21.259167,24.995867,17.999480,20.943785,1.8,27.609911,4.593965
2,Argentina,Buenos Aires,2020-01-03 00:00:00+00:00,-34.6037,-58.3816,25.117500,20.517500,22.373750,27.362978,20.149343,22.910418,0.1,24.203140,5.876242
3,Argentina,Buenos Aires,2020-01-04 00:00:00+00:00,-34.6037,-58.3816,25.617500,20.917501,23.177916,27.975208,21.116510,24.117735,0.0,20.833395,5.913107
4,Argentina,Buenos Aires,2020-01-05 00:00:00+00:00,-34.6037,-58.3816,27.167501,20.217500,24.225832,29.682999,22.054735,25.340597,0.0,18.486622,5.996509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Venezuela,Caracas,2024-04-16 00:00:00+00:00,10.4806,-66.9036,30.464000,19.014000,24.030664,34.626366,20.114017,27.005964,1.9,12.261158,5.001820
31436,Venezuela,Caracas,2024-04-17 00:00:00+00:00,10.4806,-66.9036,27.864000,20.764000,23.861917,32.559837,22.816622,27.147459,6.6,9.531505,3.589120
31437,Venezuela,Caracas,2024-04-18 00:00:00+00:00,10.4806,-66.9036,27.264000,20.914000,23.261915,31.179459,23.071089,26.784079,4.3,7.920000,2.683167
31438,Venezuela,Caracas,2024-04-19 00:00:00+00:00,10.4806,-66.9036,26.314000,21.114000,23.245253,31.100521,24.506313,27.202951,10.2,6.130579,3.454755


Observe, abaixo, que o dataframe dados_clima possui uma série de linhas com informações nulas para determinados indicadores, além de não ter, nativamente, os nomes das capitais, tão somente suas coordenadas geográficas.

In [7]:
dados_clima

,Data,Latitude,Longitude,PM10,PM2_5,Monóxido de Carbono (CO),Dióxido de Nitrogênio (NO2),Dioxido de Enxofre(SO2),Ozônio(O3)
0,2022-07-29 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-07-30 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-31 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-01 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-02 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12635,2024-04-16 00:00:00+00:00,10.4806,-66.9036,9.3,6.1,232.0,7.3,2.1,41.0
12636,2024-04-17 00:00:00+00:00,10.4806,-66.9036,28.5,19.6,510.0,18.5,3.4,36.0
12637,2024-04-18 00:00:00+00:00,10.4806,-66.9036,23.1,16.2,446.0,14.2,3.2,58.0
12638,2024-04-19 00:00:00+00:00,10.4806,-66.9036,40.7,29.1,775.0,26.5,5.6,49.0


### **Utilizando API para obter cidades**

### **API Nominatim**

A API **Nominatim** é um serviço de geocodificação e reverso-geocodificação baseado nos dados do **OpenStreetMap(OSM)**, que são constantemente atualizados por voluntários.
- Geocodificação: converte um endereço ou nome de local (ex: *"Praça da Liberdade, Belo Horizonte"* em coordenadas geográficas (latitude e longitude).
- Reverso-geocodificação: faz o oposto - converte coordenadas em um endereço legível (ex: -19.932, -43.938 -> *"Praça da Liberdade, Belo Horizonte, Brasil"*).

Entre seus principais atributos, estão a gratuidade (mas com limite de uso para evitar sobrecarga - geralmente, 1 requisição por segundo por IP), e seu caráter público. Os dados retornados das requisições são formatados como JSON, XML ou HTML, o que facilita a integração em aplicações web ou scripts Python. Além disso, a API também pode ser utilizada para criar mapas interativos, localizar endereços ou enriquecer bancos de dados com coordenadas.

Para solucionar o problema das coordenadas geográficas em dados_clima, faremos uso dessa API.

In [8]:
# Obter coordenadas únicas
coordenadas_unicas = dados_clima[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Usar a API Nominatim (OpenStreetMap)
geolocator = Nominatim(user_agent="qualidade_ar_analise")
nomes = []

print("🔎 Consultando cidades... (pode demorar um pouco)")
for i, row in coordenadas_unicas.iterrows():
    try:
        location = geolocator.reverse((row['Latitude'], row['Longitude']), language='pt')
        if location and 'address' in location.raw:
            cidade = (location.raw['address'].get('city') or
                      location.raw['address'].get('town') or
                      location.raw['address'].get('state') or
                      "Desconhecida")
        else:
            cidade = "Desconhecida"
        nomes.append(cidade)
        print(f"→ {row['Latitude']}, {row['Longitude']} → {cidade}")
    except Exception as e:
        print(f"⚠️ Erro em {i}: {e}")
        nomes.append("Erro")
    time.sleep(1)  # evita bloqueio de requisições pela API

coordenadas_unicas["Cidade"] = nomes

# Juntar as cidades ao DataFrame original
dados_clima = dados_clima.merge(coordenadas_unicas, on=["Latitude", "Longitude"], how="left")

# Converter o DataFrame para formato longo (ideal para gráficos)
dados_longos = dados_clima.melt(
    id_vars=["Cidade"],
    value_vars=['PM10', 'PM2_5', 'Monóxido de Carbono (CO)', 'Dióxido de Nitrogênio (NO2)', 'Dioxido de Enxofre(SO2)', 'Ozônio(O3)'],
    var_name="Indicador",
    value_name="Valor"
)

🔎 Consultando cidades... (pode demorar um pouco)


→ -34.6037, -58.3816 → Buenos Aires
→ -19.0196, -65.2619 → Sucre


→ -15.8267, -47.9218 → Brasília
→ -33.4378, -70.6504 → Região Metropolitana de Santiago
→ 4.711, -74.0721 → Bogotá
→ 9.9281, -84.0907 → San José
→ 23.1136, -82.3666 → La Habana
→ 18.4861, -69.9312 → São Domingos
→ -0.1807, -78.4678 → Quito
→ 13.6929, -89.2182 → São Salvador
→ 14.6349, -90.5069 → Cidade da Guatemala
→ 18.5944, -72.3074 → Porto Príncipe
→ 14.0723, -87.1921 → Tegucigalpa
→ 19.4326, -99.1332 → Cidade do México
→ 12.114, -86.2362 → Manágua


→ 8.9824, -79.5199 → Bella Vista
→ -25.2637, -57.5759 → Assunção
→ -12.0464, -77.0428 → Lima
→ -34.9011, -56.1645 → Montevidéu
→ 10.4806, -66.9036 → Caracas


O dataframe resultante, com as capitais associadas às suas respectivas coordenadas (latitude, longitude), é o seguinte:

In [9]:
dados_clima

,Data,Latitude,Longitude,PM10,PM2_5,Monóxido de Carbono (CO),Dióxido de Nitrogênio (NO2),Dioxido de Enxofre(SO2),Ozônio(O3),Cidade
0,2022-07-29 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
1,2022-07-30 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
2,2022-07-31 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
3,2022-08-01 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
4,2022-08-02 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...
12635,2024-04-16 00:00:00+00:00,10.4806,-66.9036,9.3,6.1,232.0,7.3,2.1,41.0,Caracas
12636,2024-04-17 00:00:00+00:00,10.4806,-66.9036,28.5,19.6,510.0,18.5,3.4,36.0,Caracas
12637,2024-04-18 00:00:00+00:00,10.4806,-66.9036,23.1,16.2,446.0,14.2,3.2,58.0,Caracas
12638,2024-04-19 00:00:00+00:00,10.4806,-66.9036,40.7,29.1,775.0,26.5,5.6,49.0,Caracas


### **Removendo Valores Nulos**

Agora, vamos tratar os valores nulos de ambos os dataframes. A análise visual confirmou que dados_clima possui valores faltantes, mas pode ser que isto não esteja explícito em dados_temp. Por precaução, trataremos destes casos nulos em ambas as bases.

In [10]:
dados_clima.dropna()

,Data,Latitude,Longitude,PM10,PM2_5,Monóxido de Carbono (CO),Dióxido de Nitrogênio (NO2),Dioxido de Enxofre(SO2),Ozônio(O3),Cidade
6,2022-08-04 00:00:00+00:00,-34.6037,-58.3816,9.7,6.7,174.0,22.10,8.9,21.0,Buenos Aires
7,2022-08-05 00:00:00+00:00,-34.6037,-58.3816,16.0,11.0,140.0,11.20,13.8,35.0,Buenos Aires
8,2022-08-06 00:00:00+00:00,-34.6037,-58.3816,45.5,31.7,194.0,30.00,52.7,33.0,Buenos Aires
9,2022-08-07 00:00:00+00:00,-34.6037,-58.3816,12.5,7.7,163.0,13.70,8.6,43.0,Buenos Aires
10,2022-08-08 00:00:00+00:00,-34.6037,-58.3816,9.8,6.0,111.0,9.25,7.5,45.0,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...
12635,2024-04-16 00:00:00+00:00,10.4806,-66.9036,9.3,6.1,232.0,7.30,2.1,41.0,Caracas
12636,2024-04-17 00:00:00+00:00,10.4806,-66.9036,28.5,19.6,510.0,18.50,3.4,36.0,Caracas
12637,2024-04-18 00:00:00+00:00,10.4806,-66.9036,23.1,16.2,446.0,14.20,3.2,58.0,Caracas
12638,2024-04-19 00:00:00+00:00,10.4806,-66.9036,40.7,29.1,775.0,26.50,5.6,49.0,Caracas


In [11]:
dados_temp.dropna()

,País,Cidade,Data,Latitude,Longitude,Temperatura_2m_max,Temperatura_2m_min,Temperatura_2m_media,Temperatura_aparente_max,Temperatura_aparente_min,Temperatura_aparente_media,Soma_precipitacao,Velocidade_vento_10m_max,Et0_fao_evapotranspiração
0,Argentina,Buenos Aires,2020-01-01 00:00:00+00:00,-34.6037,-58.3816,24.767500,19.267500,21.973753,25.061949,18.268500,22.049086,0.6,24.336640,4.160744
1,Argentina,Buenos Aires,2020-01-02 00:00:00+00:00,-34.6037,-58.3816,24.217500,18.317501,21.259167,24.995867,17.999480,20.943785,1.8,27.609911,4.593965
2,Argentina,Buenos Aires,2020-01-03 00:00:00+00:00,-34.6037,-58.3816,25.117500,20.517500,22.373750,27.362978,20.149343,22.910418,0.1,24.203140,5.876242
3,Argentina,Buenos Aires,2020-01-04 00:00:00+00:00,-34.6037,-58.3816,25.617500,20.917501,23.177916,27.975208,21.116510,24.117735,0.0,20.833395,5.913107
4,Argentina,Buenos Aires,2020-01-05 00:00:00+00:00,-34.6037,-58.3816,27.167501,20.217500,24.225832,29.682999,22.054735,25.340597,0.0,18.486622,5.996509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Venezuela,Caracas,2024-04-16 00:00:00+00:00,10.4806,-66.9036,30.464000,19.014000,24.030664,34.626366,20.114017,27.005964,1.9,12.261158,5.001820
31436,Venezuela,Caracas,2024-04-17 00:00:00+00:00,10.4806,-66.9036,27.864000,20.764000,23.861917,32.559837,22.816622,27.147459,6.6,9.531505,3.589120
31437,Venezuela,Caracas,2024-04-18 00:00:00+00:00,10.4806,-66.9036,27.264000,20.914000,23.261915,31.179459,23.071089,26.784079,4.3,7.920000,2.683167
31438,Venezuela,Caracas,2024-04-19 00:00:00+00:00,10.4806,-66.9036,26.314000,21.114000,23.245253,31.100521,24.506313,27.202951,10.2,6.130579,3.454755


### **Salvando as Bases Tratadas**

In [12]:
dados_clima.to_csv('LA_daily_air_quality_clean.csv', index=False)
dados_temp.to_csv('LA_daily_climate_clean.csv',index=False)

In [13]:
from google.colab import files
files.download('LA_daily_air_quality_clean.csv')
files.download('LA_daily_climate_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Feitas as modificações necessárias, estamos prontos para prosseguir para a primeira fase do projeto, que consiste na análise exploratória dos dados.